In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import sys
sys.path.append("/Users/mprange/PycharmProjects/flight_phase_seperation/")

import FlightPhaseTools as fpt

# Load data into xarray datasets using typhon FileSet objects.

In [ ]:
bahamas = xr.open_dataset('unified/bahamas_20200128_v0.4.nc')
dropsondes = xr.open_dataset('unified/dropsondes_20200128_v0.4.nc')

# Determine the timestamps of the different flight phases using the FlightPhaseTools module.

In [ ]:
takeoff = 0
touchdown = len(bahamas['time']) - 1
enter_c1 = fpt.ds_timestamp_ind(bahamas, dropsondes, 0)
exit_c1 = fpt.ds_timestamp_ind(bahamas, dropsondes, 11)
enter_c2 = fpt.ds_timestamp_ind(bahamas, dropsondes, 12)
exit_c2 = fpt.ds_timestamp_ind(bahamas, dropsondes, 23)
enter_c3 = fpt.ds_timestamp_ind(bahamas, dropsondes, 24)
exit_c3 = fpt.ds_timestamp_ind(bahamas, dropsondes, 34)
max_roll_b4_leg1 = fpt.max_timestamp_ind(bahamas, 'roll', 
          slice('2020-01-28T18:30:00', '2020-01-28T19:00:00'),
          order=500
                                        )
enter_leg1 = fpt.find_first_value_after_ts_ind(bahamas, 'roll', max_roll_b4_leg1, 0., 0.05)
max_roll_after_leg1 = fpt.max_timestamp_ind(bahamas, 'roll', 
          slice('2020-01-28T19:00:00', '2020-01-28T19:20:00'),
          order=500
         )
exit_leg1 = fpt.find_first_value_before_ts_ind(bahamas, 'roll', max_roll_after_leg1, 0., 0.05)
min_roll_b4_rad_cal = fpt.min_timestamp_ind(bahamas, 'roll', 
          slice('2020-01-28T19:00:00', '2020-01-28T19:20:00'),
          order=500
         )
enter_rad_cal = fpt.find_first_value_after_ts_ind(bahamas, 'heading', min_roll_b4_rad_cal, 0., 1.0)
max_roll_after_rad_cal = fpt.max_timestamp_ind(bahamas, 'roll', 
          slice('2020-01-28T19:15:00', '2020-01-28T19:30:00'),
          order=500
         )
exit_rad_cal = fpt.find_first_value_before_ts_ind(bahamas, 'roll', max_roll_after_rad_cal, -10., 0.1)
max_roll_b4_leg2 = fpt.max_timestamp_ind(bahamas, 'roll', 
          slice('2020-01-28T19:10:00', '2020-01-28T19:30:00'),
          order=1000
                                   )
enter_leg2 = fpt.find_first_value_after_ts_ind(bahamas, 'roll', max_roll_b4_leg2, 0., 0.05)
max_roll_after_leg2 = fpt.max_timestamp_ind(bahamas, 'roll', 
                                       slice('2020-01-28T19:30:00', '2020-01-28T20:00:00'),
                                       order=1000
                                      )
exit_leg2 = fpt.find_first_value_before_ts_ind(bahamas, 'roll', max_roll_after_leg2, 0., 0.05)
enter_c4 = fpt.ds_timestamp_ind(bahamas, dropsondes, 35)
exit_c4 = fpt.ds_timestamp_ind(bahamas, dropsondes, 45)
enter_c5 = fpt.ds_timestamp_ind(bahamas, dropsondes, 46)
exit_c5 = fpt.ds_timestamp_ind(bahamas, dropsondes, 57)
enter_c6 = fpt.ds_timestamp_ind(bahamas, dropsondes, 58)
exit_c6 = fpt.ds_timestamp_ind(bahamas, dropsondes, 69)

ts_ind_list = [takeoff, touchdown, enter_c1, exit_c1, enter_c2, 
           exit_c2, enter_c3, exit_c3, enter_leg1, 
           exit_leg1, enter_rad_cal, exit_rad_cal, 
           enter_leg2, exit_leg2, enter_c4, exit_c4, 
           enter_c5, exit_c5, enter_c6, exit_c6]
label_list = ["takeoff", "touchdown", "enter_c1", "exit_c1", 
              "enter_c2", "exit_c2", "enter_c3", "exit_c3", 
              "enter_leg1", "exit_leg1", "enter_rad_cal", 
              "exit_rad_cal", "enter_leg2", "exit_leg2", 
              "enter_c4", "exit_c4", "enter_c5", "exit_c5", 
              "enter_c6", "exit_c6"]

# Create basic timeseries of bahamas data with markers for determined timestamps to manually evaluate the validity of the timestamps.

In [ ]:
fig, ax = fpt.plot_bahamas_timeseries(bahamas, ts_ind_list, label_list)

# Create xarray datasets of the determined flight phase timestamps and store them as NetCDF files.

## Version 2

In [ ]:
time_stamps = bahamas['time'][ts_list].values.reshape((int(len(ts_ind_list) / 2), 2))
time_stamp_labels = np.array(['liftoff_touchdown', 'c1', 'c2', 'c3', 
                              'leg1', 'radar_calibration', 'leg2', 'c4', 'c5', 'c6'])
flight_phase_ds = xr.Dataset(
    coords=
    {
        'flight_phase': (time_stamp_labels),
        'start_end': np.array(['phase_start', 'phase_end'])
    },
    data_vars=
    {
        'timestamp': (('flight_phase', 'start_end'), time_stamps),
    },
    attrs=
    {
        'mission': bahamas.attrs['mission'],
        'flight_number': bahamas.attrs['flight_number'],
        'flight_date': str(pd.to_datetime(bahamas['time'][0].values).date()), 
        'contact': 'marc.prange@uni-hamburg.de',
    }
)
flight_phase_ds

In [ ]:
flight_phase_ds.to_netcdf(
    f'flight_phase_timestamps/RF_{flight_phase_ds.flight_number}_' \
    f'{flight_phase_ds.flight_date}_flight_phase_timestamps_v2.nc')